In [4]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash


--2021-12-24 19:57:31--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 19:57:33--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 19:57:34--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:44

In [5]:
!wget http://setup.johnsnowlabs.com/colab.sh 

! cat /content/colab.sh

--2021-12-24 19:58:14--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2021-12-24 19:58:15--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-12-24 19:58:16--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:44

In [6]:
import pandas as pd
import numpy as np

# 2.1 spark related
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType, StringType,StructField,StructType
#Replace part of string with another string
from pyspark.sql.functions import regexp_replace

# 2.2 Spark-nlp related
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

In [7]:
spark = sparknlp.start()

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [9]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [10]:
!ls

colab.sh  sample_data  toxic_test.snappy.parquet  toxic_train.snappy.parquet


In [11]:
!cd drive

/bin/bash: line 0: cd: drive: No such file or directory


In [12]:
pathToFoler = "/gdrive/MyDrive/Colab Notebooks/"
fileName = "train.csv"
fileName2 = "test.csv"

In [13]:
data_train = pd.read_csv(pathToFoler+fileName)
data_test = pd.read_csv(pathToFoler+fileName2)
data_test.head(5)


,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [14]:
change_val = {0: 'Not disaster related',1:'Disaster related'}
data_train['target'] = data_train['target'].replace(change_val)
#data_test['target'] = data_test['target'].replace(change_val)

In [15]:
data_train.tail(5)

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,Disaster related
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,Disaster related
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,Disaster related
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,Disaster related
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,Disaster related


In [16]:
data_train.pop('keyword')
data_train.pop('location')
data_test.pop('keyword')
data_test.pop('location')


0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
7608    NaN
7609    NaN
7610    NaN
7611    NaN
7612    NaN
Name: keyword, Length: 7613, dtype: object

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
7608    NaN
7609    NaN
7610    NaN
7611    NaN
7612    NaN
Name: location, Length: 7613, dtype: object

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3258    NaN
3259    NaN
3260    NaN
3261    NaN
3262    NaN
Name: keyword, Length: 3263, dtype: object

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
3258    NaN
3259    NaN
3260    NaN
3261    NaN
3262    NaN
Name: location, Length: 3263, dtype: object

In [19]:
df_train = spark.createDataFrame(data_train)
df_test = spark.createDataFrame(data_test)

In [20]:
df_train.show()
df_test.show()

+---+--------------------+--------------------+
| id|                text|              target|
+---+--------------------+--------------------+
|  1|Our Deeds are the...|    Disaster related|
|  4|Forest fire near ...|    Disaster related|
|  5|All residents ask...|    Disaster related|
|  6|13,000 people rec...|    Disaster related|
|  7|Just got sent thi...|    Disaster related|
|  8|#RockyFire Update...|    Disaster related|
| 10|#flood #disaster ...|    Disaster related|
| 13|I'm on top of the...|    Disaster related|
| 14|There's an emerge...|    Disaster related|
| 15|I'm afraid that t...|    Disaster related|
| 16|Three people died...|    Disaster related|
| 17|Haha South Tampa ...|    Disaster related|
| 18|#raining #floodin...|    Disaster related|
| 19|#Flood in Bago My...|    Disaster related|
| 20|Damage to school ...|    Disaster related|
| 23|      What's up man?|Not disaster related|
| 24|       I love fruits|Not disaster related|
| 25|    Summer is lovely|Not disaster r

In [21]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained("tfhub_use_lg", "en") \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("target")\
    .setMaxEpochs(5)\
    .setEnableOutputLogs(True)

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])


tfhub_use_lg download started this may take some time.
Approximate size to download 753.3 MB
[OK!]


In [22]:
pipelineModel = pipeline.fit(df_train)

In [23]:
result = pipelineModel.transform(df_test)

In [24]:
result.show()

+---+--------------------+--------------------+--------------------+--------------------+
| id|                text|            document| sentence_embeddings|               class|
+---+--------------------+--------------------+--------------------+--------------------+
|  0|Just happened a t...|[[document, 0, 33...|[[sentence_embedd...|[[category, 0, 33...|
|  2|Heard about #eart...|[[document, 0, 63...|[[sentence_embedd...|[[category, 0, 63...|
|  3|there is a forest...|[[document, 0, 95...|[[sentence_embedd...|[[category, 0, 95...|
|  9|Apocalypse lighti...|[[document, 0, 39...|[[sentence_embedd...|[[category, 0, 39...|
| 11|Typhoon Soudelor ...|[[document, 0, 44...|[[sentence_embedd...|[[category, 0, 44...|
| 12|We're shaking...I...|[[document, 0, 33...|[[sentence_embedd...|[[category, 0, 33...|
| 21|They'd probably s...|[[document, 0, 71...|[[sentence_embedd...|[[category, 0, 71...|
| 22|   Hey! How are you?|[[document, 0, 16...|[[sentence_embedd...|[[category, 0, 16...|
| 27|    W

In [25]:
result_df = result.select('text',"class.result").toPandas()

In [26]:
result_df.head(50)

,text,result
0,Just happened a terrible car crash,[Not disaster related]
1,"Heard about #earthquake is different cities, s...",[Disaster related]
2,"there is a forest fire at spot pond, geese are...",[Disaster related]
3,Apocalypse lighting. #Spokane #wildfires,[Disaster related]
4,Typhoon Soudelor kills 28 in China and Taiwan,[Disaster related]
5,We're shaking...It's an earthquake,[Disaster related]
6,They'd probably still show more life than Arse...,[Not disaster related]
7,Hey! How are you?,[Not disaster related]
8,What a nice hat?,[Not disaster related]
9,Fuck off!,[Not disaster related]


**MultiLabel Classifier**

In [27]:
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_train.snappy.parquet'
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_test.snappy.parquet'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2702k  100 2702k    0     0  1186k      0  0:00:02  0:00:02 --:--:-- 1186k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  289k  100  289k    0     0   167k      0  0:00:01  0:00:01 --:--:--  167k


In [28]:
trainDataset = spark.read.parquet("/content/toxic_train.snappy.parquet").repartition(120)
testDataset = spark.read.parquet("/content/toxic_test.snappy.parquet").repartition(10)

Multilabel Classifier with Bert Embeddings

In [30]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")\
      .setCleanupMode("shrink")

bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset
multiClassifier = MultiClassifierDLApproach()\
      .setInputCols("sentence_embeddings")\
      .setOutputCol("category")\
      .setLabelColumn("labels")\
      .setBatchSize(128)\
      .setMaxEpochs(5)\
      .setLr(1e-3)\
      .setThreshold(0.5)\
      .setShufflePerEpoch(False)\
      .setEnableOutputLogs(True)\
      .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        bert_sent,
        multiClassifier
    ])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[OK!]


In [31]:
%%time
pipelineModel = pipeline.fit(trainDataset)

CPU times: user 20.2 s, sys: 2.05 s, total: 22.3 s
Wall time: 59min 17s


In [32]:
preds = pipelineModel.transform(testDataset)

In [33]:
preds_df = preds.select('text','labels',"category.result").toPandas()

In [34]:
preds_df.head(10)

,text,labels,result
0,Vegan \n\nWhat in the hell does all that junk ...,[toxic],"[toxic, obscene]"
1,Fight Club! F**k Yeeaaaaahh!!!,"[toxic, obscene]","[toxic, obscene]"
2,"""\n\n Little quick on the trigger, ain't'cha b...",[toxic],[toxic]
3,Your user page indicates you're a left-wing li...,"[toxic, obscene, insult]",[toxic]
4,""" See all the many Google links, titled"""" Wik...",[toxic],[toxic]
5,"""\n\n LOL \n\nLOL. Seriously, """"BryanFromPalat...",[toxic],"[toxic, obscene]"
6,is it because it is of my naked mum having sex...,"[toxic, severe_toxic, obscene, insult, identit...","[toxic, severe_toxic, insult, obscene]"
7,")\na cowards site, that must stop changing thi...","[toxic, obscene]",[toxic]
8,"blow me, criticism IS constructive. \n\nBlow m...","[toxic, obscene, insult]","[toxic, insult, obscene]"
9,On account of the project deciding to ignore h...,"[toxic, obscene, insult]",[toxic]


Saving & loading back the trained model

In [36]:
pipelineModel.stages

[DocumentAssembler_fd4ee7f1d52c,
 BERT_SENTENCE_EMBEDDINGS_3608c0d843af,
 MultiClassifierDLModel_74c72338e21b]

In [37]:
# Save the Multilabel Classifier Model
pipelineModel.stages[2].write().overwrite().save('MultilabelClfBert')

In [38]:
# Load back  saved Multilabel Classifier Model
MultilabelClfModel = MultiClassifierDLModel.load('MultilabelClfBert')

In [39]:
# Generate prediction Pipeline with loaded Model 
ld_pipeline = Pipeline(stages=[document, bert_sent, MultilabelClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

In [40]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(testDataset)

In [41]:
ld_preds_df = ld_preds.select('text','labels',"category.result").toPandas()

In [42]:
ld_preds_df.head(10)

,text,labels,result
0,Vegan \n\nWhat in the hell does all that junk ...,[toxic],"[toxic, obscene]"
1,Fight Club! F**k Yeeaaaaahh!!!,"[toxic, obscene]","[toxic, obscene]"
2,"""\n\n Little quick on the trigger, ain't'cha b...",[toxic],[toxic]
3,Your user page indicates you're a left-wing li...,"[toxic, obscene, insult]",[toxic]
4,""" See all the many Google links, titled"""" Wik...",[toxic],[toxic]
5,"""\n\n LOL \n\nLOL. Seriously, """"BryanFromPalat...",[toxic],"[toxic, obscene]"
6,is it because it is of my naked mum having sex...,"[toxic, severe_toxic, obscene, insult, identit...","[toxic, severe_toxic, insult, obscene]"
7,")\na cowards site, that must stop changing thi...","[toxic, obscene]",[toxic]
8,"blow me, criticism IS constructive. \n\nBlow m...","[toxic, obscene, insult]","[toxic, insult, obscene]"
9,On account of the project deciding to ignore h...,"[toxic, obscene, insult]",[toxic]
